In [ ]:
import numpy as np
from numpy.fft import fft
import matplotlib.pyplot as plt
from scipy.stats import norm

# Market and option parameters
S0 = 100       # Initial stock price
K = np.linspace(60, 140, 81)  # Strike prices from 60 to 140
T = 0.1        # Time to maturity
r = 0.05       # Risk-free rate
q = 0.02       # Dividend yield
sigma = 0.2    # Volatility

# Carr-Madan method parameters
alpha = 1.5    # Damping factor (> 0)
N = 2**16      # Number of FFT points (must be even for Simpson's Rule)
eta = 0.05     # Spacing in the frequency domain (smaller for higher accuracy)

# Ensure N is even for Simpson's Rule
if N % 2 != 0:
    N += 1

# Compute the grid in the frequency domain (v)
v = np.arange(N) * eta  # v_j = j * eta

# Compute the grid in the log-strike domain (k)
lambda_ = (2 * np.pi) / (N * eta)
b = N * lambda_ / 2     # Upper limit for k
k = -b + lambda_ * np.arange(N)  # k_i = -b + i * lambda

def phi(u):
    """Characteristic function of log(S_T) under the Black-Scholes model."""
    mu = np.log(S0) + (r -q- 0.5 * sigma**2) * T
    variance = sigma**2 * T
    return np.exp(1j * u * mu - 0.5 * variance * u**2)

# Adjustment to the characteristic function for damping
u = v - (alpha + 1) * 1j
psi = np.exp(-r * T) * phi(u) / (alpha**2 + alpha - v**2 + 1j * (2 * alpha + 1) * v)

#   Simpson's Rule weights
w = np.ones(N)
w[0] = w[-1] = 1
w[1:-1:2] = 4  # Weights for odd indices
w[2:-1:2] = 2  # Weights for even indices
w = w * (eta / 3)

# Function to be transformed with   weights
fft_input = psi * np.exp(1j * v * b) * w

# Apply the FFT
fft_output = fft(fft_input)

# Compute the option prices
C_k = np.exp(-alpha * k) / np.pi * np.real(fft_output)

# Convert strikes to log-strikes
log_K = np.log(K)

# Interpolate the option prices at desired strikes
call_prices_fft = np.interp(log_K, k, C_k)

# Black-Scholes formula for European call options
def black_scholes_call(S0, K, T, r, sigma, q):
    """Black-Scholes formula for European call options."""
    d1 = (np.log(S0 / K) + (r -q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-q * T)*S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Compute Black-Scholes prices
call_prices_bs = black_scholes_call(S0, K, T, r, sigma, q)

# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(K, call_prices_fft, label='Carr-Madan FFT Price', linestyle='--')
plt.plot(K, call_prices_bs, label='Black-Scholes Price', alpha=0.7)
plt.xlabel('Strike Price K')
plt.ylabel('Option Price')
plt.title('Comparison of Carr-Madan FFT and Black-Scholes Prices')
plt.legend()
plt.grid(True)
plt.show()

# Optionally, plot the absolute difference
plt.figure(figsize=(10, 6))
plt.plot(K, np.abs(call_prices_fft - call_prices_bs), label='Absolute Difference')
plt.xlabel('Strike Price K')
plt.ylabel('Price Difference')
plt.title('Absolute Difference between Carr-Madan FFT and Black-Scholes Prices')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

# -----------------------------
# 1. Define Heston Model Parameters
# -----------------------------

# Market parameters
S0 = 100        # Initial stock price
r = 0.05        # Risk-free interest rate
q = 0.02        # Dividend yield
T = 1.0         # Time to maturity (in years)

# Heston model parameters
kappa = 1.5     # Speed of mean reversion of variance
theta = 0.04    # Long-term variance
sigma_v = 0.3   # Volatility of variance (vol of vol)
rho = -0.7      # Correlation between asset and variance
v0 = 0.04       # Initial variance

# Carr-Madan parameters (Enhanced)
alpha = 1.5             # Dampening factor (> 0)
N = 2**16             # Number of FFT points (power of 2)
eta = 0.25              # Spacing of the grid in Fourier space
lambda_ = (2 * np.pi) / (N * eta)  # Spacing in log-strike space
b = N * lambda_ / 2  # Upper bound of log-strike

# Monte Carlo simulation parameters (Enhanced)
num_paths = 100000     # Number of simulated paths
num_steps = 500        # Number of time steps

# -----------------------------
# 2. Define the Heston Characteristic Function
# -----------------------------

def heston_char_func(u, T, kappa, theta, sigma_v, rho, v0, r, q, S0):
    """
    Characteristic function of log(S_T) under the Heston model.
    """
    i = 1j
    sigma = sigma_v

    d = np.sqrt((rho * sigma * i * u - kappa) ** 2 + sigma ** 2 * (i * u + u ** 2))
    g = (kappa - rho * sigma * i * u - d) / (kappa - rho * sigma * i * u + d)

    exp_dT = np.exp(-d * T)

    C = (np.log(S0)+(r-q)*T) * i * u  + (kappa * theta / sigma ** 2) * ((kappa - rho * sigma * i * u - d) * T - 2 * np.log((1 - g * exp_dT) / (1 - g)))

    D = ((kappa - rho * sigma * i * u - d) / sigma ** 2) * (
        (1 - exp_dT) / (1 - g * exp_dT)
    )

    phi = np.exp(C + D * v0)
    return phi

# -----------------------------
# 3. Carr-Madan Option Pricing Function (Enhanced)
# -----------------------------

def carr_madan_call_price(S0, r, q, T, kappa, theta, sigma_v, rho, v0, alpha, N, eta):
    """
    Calculate the European call option prices using Carr-Madan method under Heston model.
    """
    i = 1j
    # Step 1: Compute the modified characteristic function
    u = np.arange(N) * eta  # u values

    # Avoid division by zero
    u[0] = 1e-22

    # Characteristic function with dampening factor
    phi = np.exp(-r * T) * heston_char_func(u - (alpha + 1) * i, T, kappa, theta, sigma_v, rho, v0, r, q, S0) \
        / (alpha ** 2 + alpha - u ** 2 + i * (2 * alpha + 1) * u)

    # Step 2: Apply Simpson's Rule weights
    SimpsonW = np.ones(N)
    SimpsonW[0] = 1
    SimpsonW[1:N-1:2] = 4
    SimpsonW[2:N-2:2] = 2
    SimpsonW[-1] = 1
    SimpsonW = SimpsonW * (eta / 3)

    # Step 3: Compute the FFT
    x = phi * np.exp(i * b * u) * SimpsonW
    fft_x = np.fft.fft(x)
    fft_x = np.real(fft_x)

    # Step 4: Recover the call prices
    k = -b + np.arange(N) * lambda_
    K = np.exp(k)
    call_prices = np.exp(-alpha * k) / np.pi * fft_x

    return K, call_prices

# -----------------------------
# 4. Monte Carlo Simulation Function (Enhanced)
# -----------------------------

def heston_simulation(S0, v0, r, q, T, kappa, theta, sigma_v, rho, num_paths, num_steps):
    """
    Simulate asset price paths under the Heston model using antithetic variates.
    """
    dt = T / num_steps
    sqrt_dt = np.sqrt(dt)
    np.random.seed(0)  # For reproducibility

    num_paths_half = num_paths // 2

    # Initialize arrays
    S = np.zeros((num_paths, num_steps + 1))
    v = np.zeros((num_paths, num_steps + 1))

    # Set initial values
    S[:, 0] = S0
    v[:, 0] = v0

    # Cholesky decomposition for correlated Brownian motions
    cov_matrix = np.array([[1.0, rho], [rho, 1.0]])
    L = np.linalg.cholesky(cov_matrix)

    for t in range(1, num_steps + 1):
        # Generate standard normal random variables
        Z = np.random.randn(num_paths_half, 2)
        Z_antithetic = -Z  # Antithetic variates
        Z_full = np.vstack((Z, Z_antithetic))

        # Correlated random variables
        dW = np.dot(Z_full, L.T) * sqrt_dt

        # Update variance process using full truncation Euler scheme
        v_prev = v[:, t - 1]
        sqrt_v_prev = np.sqrt(np.maximum(v_prev, 0))
        dv = kappa * (theta - np.maximum(v_prev, 0)) * dt + sigma_v * sqrt_v_prev * dW[:, 1]
        v[:, t] = np.maximum(v_prev + dv, 0)  # Ensure variance is non-negative

        # Update asset price
        S_prev = S[:, t - 1]
        S[:, t] = S_prev * np.exp((r - q - 0.5 * np.maximum(v_prev, 0)) * dt + sqrt_v_prev * dW[:, 0])

    # Return asset prices at maturity
    return S[:, -1]

def heston_option_pricing(S0, v0, r, q, T, kappa, theta, sigma_v, rho, num_paths, num_steps, K_values):
    """
    Price European call options using Monte Carlo simulation under the Heston model.
    """
    # Simulate asset prices at maturity
    S_T = heston_simulation(S0, v0, r, q, T, kappa, theta, sigma_v, rho, num_paths, num_steps)

    # Initialize array for call prices
    call_prices = np.zeros_like(K_values)

    # Discount factor
    discount_factor = np.exp(-r * T)

    # Calculate option prices for each strike price
    payoffs_matrix = np.maximum(S_T[:, np.newaxis] - K_values, 0)
    call_prices = discount_factor * np.mean(payoffs_matrix, axis=0)

    return call_prices

# -----------------------------
# 5. Calculate Option Prices Using Both Methods
# -----------------------------

# Carr-Madan method
start_time_cm = time.time()
K_cm, call_prices_cm = carr_madan_call_price(
    S0, r, q, T, kappa, theta, sigma_v, rho, v0, alpha, N, eta
)
end_time_cm = time.time()
time_cm = end_time_cm - start_time_cm

# Select strike prices between 60 and 140
K_indices = np.where((K_cm >= 60) & (K_cm <= 140))
K_cm = K_cm[K_indices]
call_prices_cm = call_prices_cm[K_indices]

# Monte Carlo simulation
K_mc = np.linspace(60, 140, 81)
start_time_mc = time.time()
call_prices_mc = heston_option_pricing(
    S0, v0, r, q, T, kappa, theta, sigma_v, rho, num_paths, num_steps, K_mc
)
end_time_mc = time.time()
time_mc = end_time_mc - start_time_mc

# -----------------------------
# 6. Compute Price Differences
# -----------------------------

# Interpolate Carr-Madan prices to match K_mc
call_prices_cm_interp = np.interp(K_mc, K_cm, call_prices_cm)

# Compute price differences
price_differences = np.abs(call_prices_cm_interp - call_prices_mc)
max_diff = np.max(price_differences)

# -----------------------------
# 7. Plot the Option Prices
# -----------------------------

plt.figure(figsize=(10, 6))
plt.plot(K_mc, call_prices_cm_interp, label='Carr-Madan Method', color='blue', linewidth=2)
plt.plot(K_mc, call_prices_mc, label='Monte Carlo Simulation', color='red', linestyle='--', linewidth=2)
plt.xlabel('Strike Price K', fontsize=14)
plt.ylabel('Option Price', fontsize=14)
plt.title('European Call Option Prices under Heston Model', fontsize=16)
plt.grid(True)
plt.legend(fontsize=12)
plt.show()


# -----------------------------
# 8. Plot Price Differences
# -----------------------------

plt.figure(figsize=(10, 6))
plt.plot(K_mc, price_differences, label='Price Difference (Carr-Madan - MC)', color='green', linewidth=2)
plt.xlabel('Strike Price K', fontsize=14)
plt.ylabel('Price Difference', fontsize=14)
plt.title('Price Differences between Carr-Madan and Monte Carlo Methods', fontsize=16)
plt.grid(True)
plt.legend(fontsize=12)
plt.show()

# -----------------------------
# 9. Report Computation Times and Maximum Price Difference
# -----------------------------

print(f"Computation Time (Carr-Madan Method): {time_cm:.2f} seconds")
print(f"Computation Time (Monte Carlo Simulation): {time_mc:.2f} seconds")
print(f"Maximum Price Difference: {max_diff:.6f}")

In [ ]:
import numpy as np
from numpy.fft import fft
import matplotlib.pyplot as plt
import time

# Market and option parameters
S0 = 100        # Initial stock price
r = 0.05        # Risk-free rate
q = 0.0         # Dividend yield
T = 1.0         # Time to maturity

# Variance Gamma parameters
sigma = 0.12    # Volatility parameter
nu = 0.2        # Variance rate of the Gamma process
theta = -0.14   # Drift parameter

# Strike prices
K = np.linspace(60, 140, 81)  # Strike prices from 60 to 140

# Carr-Madan method parameters
alpha = 5.0     # Damping factor (> 0), increased for better convergence
N = 2**14       # Number of FFT points (must be even for Simpson's Rule)
eta = 0.05      # Spacing in the frequency domain, decreased for better accuracy

# Ensure N is even for Simpson's Rule
if N % 2 != 0:
    N += 1

# Start timing the Carr-Madan method
start_time_fft = time.time()

# Compute the grid in the frequency domain (v)
v = np.arange(N) * eta  # v_j = j * eta

# Compute the grid in the log-strike domain (k)
lambda_ = (2 * np.pi) / (N * eta)
b = N * lambda_ / 2     # Upper limit for k
k = -b + lambda_ * np.arange(N)  # k_i = -b + i * lambda

def phi(u):
    """
    characteristic function of log(S_T) under the Variance Gamma model.
    """
    i = 1j
    omega = (1 / nu) * np.log(1 - theta * nu - 0.5 * sigma**2 * nu)
    drift = r - q - omega  
    phi_u = np.exp(i * u * (np.log(S0) + drift * T)) * \
            (1 - i * theta * nu * u + 0.5 * sigma**2 * nu * u**2) ** (-T / nu)
    return phi_u

# Adjusted characteristic function
u = v - (alpha + 1) * 1j
psi = (np.exp(-r * T) * phi(u)) / (alpha**2 + alpha - v**2 + 1j * (2 * alpha + 1) * v)

#   Simpson's Rule weights
w = np.ones(N)
w[0] = w[-1] = 1
w[1:-1:2] = 4  # Weights for odd indices
w[2:-1:2] = 2  # Weights for even indices
w = w * (eta / 3)

# Function to be transformed
fft_input = psi * np.exp(-1j * v * b) * w  #   sign in exponential term

# Apply the FFT
fft_output = fft(fft_input)

# Compute the option prices
C_k = np.exp(-alpha * k) / np.pi * np.real(fft_output)

# Convert log-strike grid to strike prices
K_fft = np.exp(k)

# Interpolate the option prices at desired strikes
call_prices_fft = np.interp(K, K_fft, C_k)

# End timing the Carr-Madan method
end_time_fft = time.time()
time_fft = end_time_fft - start_time_fft

# Monte Carlo simulation with variance reduction
def vg_process_mc(S0, T, r, q, sigma, theta, nu, M):
    """
    Monte Carlo simulation for the Variance Gamma process with antithetic variates.
    """
    np.random.seed(0)  # For reproducibility

    # Simulate Gamma process
    N_sim = T / nu
    G = np.random.gamma(N_sim, nu, M)

    # Antithetic variates for Z
    Z = np.random.randn(M)
    Z_antithetic = -Z

    # VG process for original Z
    X = theta * G + sigma * np.sqrt(G) * Z
    # VG process for antithetic Z
    X_antithetic = theta * G + sigma * np.sqrt(G) * Z_antithetic

    # Adjusted drift to ensure martingale
    omega = (1 / nu) * np.log(1 - theta * nu - 0.5 * sigma**2 * nu)
    drift = (r - q - omega) * T  #   drift

    # Simulate stock prices at maturity
    ST = S0 * np.exp(drift + X)
    ST_antithetic = S0 * np.exp(drift + X_antithetic)

    # Combine the two sets of simulated prices
    ST_combined = np.concatenate((ST, ST_antithetic))

    return ST_combined

M = 500000  # Number of simulations (total simulations will be 1,000,000 with antithetic variates)

# Start timing the Monte Carlo method
start_time_mc = time.time()

# Simulate stock prices
ST_combined = vg_process_mc(S0, T, r, q, sigma, theta, nu, M)

# Compute payoffs
payoffs = np.maximum(ST_combined[:, np.newaxis] - K, 0)

# Discounted option prices
call_prices_mc = np.exp(-r * T) * np.mean(payoffs, axis=0)

# End timing the Monte Carlo method
end_time_mc = time.time()
time_mc = end_time_mc - start_time_mc

# Compute the price differences
price_differences = call_prices_fft - call_prices_mc


# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(K, call_prices_fft, label='Carr-Madan FFT Price (VG)', linestyle='--')
plt.plot(K, call_prices_mc, label='Monte Carlo Price (VG)', alpha=0.7)
plt.xlabel('Strike Price K')
plt.ylabel('Option Price')
plt.title('Comparison of Carr-Madan FFT and Monte Carlo Prices (VG)')
plt.legend()
plt.grid(True)
plt.show()

# Plot the differences in prices
plt.figure(figsize=(10, 6))
plt.plot(K, price_differences, label='Price Difference (FFT - MC)')
plt.xlabel('Strike Price K')
plt.ylabel('Price Difference')
plt.title('Difference in Option Prices between Carr-Madan FFT and Monte Carlo (VG)')
plt.legend()
plt.grid(True)
plt.show()

# Calculate the maximum absolute difference
max_difference = np.max(np.abs(price_differences))
print(f"Maximum Absolute Price Difference: {max_difference:.6f}")

# Print computational times
print(f"Computational Time (Carr-Madan FFT): {time_fft:.6f} seconds")
print(f"Computational Time (Monte Carlo): {time_mc:.6f} seconds")


In [ ]:
import numpy as np
from numpy.fft import fft
from scipy.stats import norm
import matplotlib.pyplot as plt
import time

# Market and option parameters
S0 = 100       # Initial stock price
r = 0.05       # Risk-free rate
q = 0.0        # Dividend yield
T = 0.1        # Time to maturity

# Volatility
sigma = 0.2    # Volatility of diffusion component

# Jump parameters
lam = 0.1      # Jump intensity (lambda)
mu_j = -0.1    # Mean of jump size (mu_J)
sigma_j = 0.3  # Standard deviation of jump size (sigma_J)

# Calculated parameter kappa
kappa = np.exp(mu_j + 0.5 * sigma_j**2) - 1

# Strike prices
K = np.linspace(60, 140, 81)  # Strike prices from 60 to 140

# Carr-Madan method parameters
alpha = 1.5    # Damping factor (> 0)
N = 2**16      # Number of FFT points (must be even for Simpson's Rule)
eta = 0.05     # Spacing in the frequency domain (smaller for higher accuracy)

# Ensure N is even for Simpson's Rule
if N % 2 != 0:
    N += 1

# Start timing the Carr-Madan method
start_time_fft = time.time()

# Compute the grid in the frequency domain (v)
v = np.arange(N) * eta  # v_j = j * eta

# Compute the grid in the log-strike domain (k)
lambda_ = (2 * np.pi) / (N * eta)
b = N * lambda_ / 2     # Upper limit for k
k = -b + lambda_ * np.arange(N)  # k_i = -b + i * lambda

def phi(u):
    """
    Characteristic function of log(S_T) under the Merton Jump-Diffusion model.
    """
    i = 1j
    mu = np.log(S0) + (r - q - lam * kappa - 0.5 * sigma**2) * T
    variance = sigma**2 * T
    phi_jump = lam * T * (np.exp(i * u * mu_j - 0.5 * sigma_j**2 * u**2) - 1)
    cf = np.exp(i * u * mu - 0.5 * variance * u**2 + phi_jump)
    return cf

# Adjusted characteristic function
u = v - (alpha + 1) * 1j
psi = np.exp(-r * T) * phi(u) / (alpha**2 + alpha - v**2 + 1j * (2 * alpha + 1) * v)

#   Simpson's Rule weights
w = np.ones(N)
w[0] = w[-1] = 1
w[1:-1:2] = 4  # Weights for odd indices
w[2:-1:2] = 2  # Weights for even indices
w = w * (eta / 3)

# Function to be transformed
fft_input = psi * np.exp(-1j * v * b) * w

# Apply the FFT
fft_output = fft(fft_input)

# Compute the option prices
C_k = np.exp(-alpha * k) / np.pi * np.real(fft_output)

# Convert log-strike grid to strike prices
K_fft = np.exp(k)

# Interpolate the option prices at desired strikes
call_prices_fft = np.interp(K, K_fft, C_k)

# End timing the Carr-Madan method
end_time_fft = time.time()
time_fft = end_time_fft - start_time_fft

# Monte Carlo simulation
def merton_jump_diffusion_mc(S0, K, T, r, q, sigma, lam, mu_j, sigma_j, M):
    """
    Monte Carlo simulation for European call option under Merton jump-diffusion model.
    """
    np.random.seed(0)  # For reproducibility
    drift = (r - q - lam * kappa - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * np.random.randn(M)

    # Simulate number of jumps for each path
    N_jumps = np.random.poisson(lam * T, M)

    # Simulate jump sizes
    sum_Y = np.zeros(M)
    idx = np.where(N_jumps > 0)[0]
    for i in idx:
        Y_i = np.random.normal(mu_j, sigma_j, N_jumps[i])
        sum_Y[i] = np.sum(Y_i)

    # Compute log stock price at maturity
    ln_ST = np.log(S0) + drift + diffusion + sum_Y

    ST = np.exp(ln_ST)

    # Compute payoff for each simulation
    payoffs = np.maximum(ST[:, np.newaxis] - K, 0)

    # Discount back
    C_MC = np.exp(-r * T) * np.mean(payoffs, axis=0)

    return C_MC

M = 100000  # Number of simulations

# Start timing the Monte Carlo method
start_time_mc = time.time()
call_prices_mc = merton_jump_diffusion_mc(S0, K, T, r, q, sigma, lam, mu_j, sigma_j, M)
end_time_mc = time.time()
time_mc = end_time_mc - start_time_mc

# Compute the price differences
price_differences = call_prices_fft - call_prices_mc


# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(K, call_prices_fft, label='Carr-Madan FFT Price ', linestyle='--')
plt.plot(K, call_prices_mc, label='Monte Carlo Price (JD)', alpha=0.7)
plt.xlabel('Strike Price K')
plt.ylabel('Option Price')
plt.title('Comparison of Carr-Madan FFT and Monte Carlo Prices (JD)')
plt.legend()
plt.grid(True)
plt.show()

# Plot the differences in prices
plt.figure(figsize=(10, 6))
plt.plot(K, price_differences, label='Price Difference (FFT - MC)')
plt.xlabel('Strike Price K')
plt.ylabel('Price Difference')
plt.title('Difference in Option Prices between Carr-Madan FFT and Monte Carlo (JD)')
plt.legend()
plt.grid(True)
plt.show()


# Print computational times
print(f"Computational Time (Carr-Madan FFT): {time_fft:.6f} seconds")
print(f"Computational Time (Monte Carlo): {time_mc:.6f} seconds")